In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from vitascopic_nca.trainer import Trainer
from vitascopic_nca.config import DefaultTrainerConfig
from tqdm.auto import tqdm
from IPython.display import display, clear_output
import panel as pn

pn.extension()

In [3]:
config = DefaultTrainerConfig()
trainer = Trainer(config)
trainer.sanity_check()

tensor(0., device='cuda:0')
Sanity check loss: 1.1324973106384277


In [4]:
with torch.no_grad():
    info = trainer.optim_step(steps=10)
    display(trainer.display_optim_step(info))

tensor(0., device='cuda:0')


BokehModel(combine_events=True, render_bundle={'docs_json': {'65cd9a8f-f799-4ceb-be46-45a0eff46bd3': {'version…

In [5]:
pbar = tqdm(range(50_000))
for _ in pbar:
    info = trainer.optim_step(steps=(20, 105))
    pbar.set_description(f"Loss: {info['loss']:.4f}")

    if trainer.learning_steps % 250 == 1:
        with torch.no_grad():
            info = trainer.optim_step(steps=110)

        clear_output(wait=True)
        display(pbar.container)
        display(trainer.display_optim_step(info))
        trainer.save_checkpoint()

Loss: 0.9624:   0%|          | 0/50000 [00:00<?, ?it/s]

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7213d21-149c-48f8-9041-82ca4ef77e9f': {'version…

tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., device='cuda:0', grad_fn=<MulBackward0>)
tensor(0., d

KeyboardInterrupt: 